In [1]:
import fast_stats
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [2]:
pred = np.random.randint(0, 2, (10, 512, 512)).flatten()
actual = np.random.randint(0, 2, (10, 512, 512)).flatten()

In [3]:
%%timeit
_ = precision_score(actual, pred)

600 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
_ = recall_score(actual, pred)

596 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
_ = f1_score(actual, pred)

603 ms ± 3.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# don't actually need to flatten them for fast-stats
pred = np.random.randint(0, 2, (10, 512, 512))
actual = np.random.randint(0, 2, (10, 512, 512))

In [7]:
%%timeit
_ = fast_stats.precision(actual, pred)

24.1 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
_ = fast_stats.recall(actual, pred)

24.3 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
_ = fast_stats.f1_score(actual, pred)

24.2 ms ± 388 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
assert np.allclose(
    fast_stats.precision(actual.flatten(), pred.flatten()),
    precision_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.recall(actual.flatten(), pred.flatten()),
    recall_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.f1_score(actual.flatten(), pred.flatten()),
    f1_score(actual.flatten(), pred.flatten())
)